In [1]:
import json
import ollama
import openai

client = openai.OpenAI()

In [6]:
atom_schema = {
    "type": "object",
    "properties": {
        "atoms": {
            "type": "array",
            "description": "Unique atomic propositions extracted from the text.",
            "items": {
                "type": "object",
                "properties": {
                    "id": { "type": "string", "description": "e.g., a1, a2" },
                    "text": { "type": "string", "description": "The literal text of the proposition." },
                    "kb_type": { "type": "string", "enum": ["axiom", "premise"] }
                },
                "required": ["id", "text", "kb_type"],
                "additionalProperties": False
            }
        }
    },
    "required": ["atoms"],
    "additionalProperties": False
}

In [3]:
ATOM_PROMPT = ""
with open("atom_prompt.txt", "r", encoding="utf-8") as f:
    ATOM_PROMPT = f.read()

In [7]:
def respond(text: str, system_prompt: str, schema: dict):
    response = client.responses.create(
        model="gpt-4o-mini",
        temperature=0,
        input=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": text,
            }
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "argument_framework",
                "strict": True,
                "schema": schema
            }
        }
    )

    return response.output_text.strip()

In [9]:
prompts=[]
with open("prompts.txt", "r") as f:
    prompts = f.read()
for i in prompts.split("@"):
    print("Prompt:", i)
    print("Atoms:", respond(i, ATOM_PROMPT, atom_schema))
    

Prompt: If a creature is a bird, it has wings.  
This creature has wings.  
Therefore, this creature is a bird.

Atoms: {"atoms":[{"id":"a1","text":"A creature is a bird.","kb_type":"premise"},{"id":"a2","text":"A creature has wings.","kb_type":"premise"},{"id":"a3","text":"This creature has wings.","kb_type":"premise"},{"id":"a4","text":"This creature is a bird.","kb_type":"premise"}]}
Prompt: 
If it rains, the ground gets wet.  
It is not raining.  
Therefore, the ground is not wet.

Atoms: {"atoms":[{"id":"a1","text":"It rains.","kb_type":"premise"},{"id":"a2","text":"The ground gets wet.","kb_type":"premise"},{"id":"a3","text":"It is not raining.","kb_type":"premise"},{"id":"a4","text":"The ground is not wet.","kb_type":"premise"}]}
Prompt: 
If a device is encrypted and password-protected, it is secure.

Atoms: {"atoms":[{"id":"a1","text":"A device is encrypted.","kb_type":"premise"},{"id":"a2","text":"A device is password-protected.","kb_type":"premise"},{"id":"a3","text":"A devic